# Distribution of influences

To get a grip on how much randomness influences the results, we run the same model lots of times and compare the results.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules and set parameters for simulations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.15 valittaa paljon)
# ei taida toimia piilottaminen
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [2]:
# parameters for the simulation
# episode = 51 / 205 timesteps (1y/3m timestep)
pop_size=20_000 # size of the population to be simulated
size1=6_000_000 #0_000 # number of timesteps in phase 1 training (callback not used)
size2=100 #0_000 # number of timesteps in phase 2 training (callback is used to save the best results)
size3=100 # number of timesteps in phase 1 training (callback not used) for policy changes
batch1=1_00 # size of minibatch in phase 1 as number of episodes
batch2=9_00  # size of minibatch in phase 1 as number of episodes
callback_minsteps=batch2 # how many episodes callback needs 
deterministic=False # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
randomness=True # include externally given, random state-transitions (parental leaves, disability, lay-offs) 
pinkslip=True # include lay-offs at 5 percent level each year
rlmodel='acktr' # use ACKTR algorithm
twostage=False # ajataan kahdessa vaiheessa vai ei
perusmalli='best/malli_perus_final'


# Baseline

Lasketaan työllisyysasteet nykymallissa.

In [5]:
cc1=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc1.explain()
cc1.run_distrib(n=100,startn=18,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli,results='results/distrib_base',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True

Parameters of lifecycle:
timestep 0.25
gamma 0.92 per anno
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 None
ansiopvraha_kesto400 None
ansiopvraha_toe None
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki None
include_pinkslip True

computing 18
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3171     |
| nupdates           | 1        |
| policy_entropy     | 1.02     |
| policy_loss        | -0.0121  |
| total_timesteps    | 0        |
| value

---------------------------------
| explained_variance | 0.999    |
| fps                | 3412     |
| nupdates           | 250      |
| policy_entropy     | 1.02     |
| policy_loss        | -0.00936 |
| total_timesteps    | 5432433  |
| value_loss         | 0.0123   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 3408     |
| nupdates           | 260      |
| policy_entropy     | 1.01     |
| policy_loss        | -0.00784 |
| total_timesteps    | 5650603  |
| value_loss         | 0.0113   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 3407     |
| nupdates           | 270      |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0187  |
| total_timesteps    | 5868773  |
| value_loss         | 0.017    |
---------------------------------
done
simulating  best/malli_perus_final_v118
predicting...


computing 19
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3099     |
| nupdates           | 1        |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0134  |
| total_timesteps    | 0        |
| value_loss         | 0.00928  |
---------------------------------
---------------------------------
| explained_variance | 0.79     |
| fps                | 3409     |
| nupdates           | 10       |
| policy_entropy     | 0.977    |
| policy_loss        | -0.503   |
| total_timesteps    | 196353   |
| value_loss         | 2.87     |
---------------------------------
---------------------------------
| explained_variance | 0.939    |
| fps                | 3341     |
| nupdates           | 20       |
| policy_entropy     | 1.01     |
| policy_loss        | 0.201    |
| total_timesteps    | 414523   |
| value_loss         | 0.777    |
-------------------------------

---------------------------------
| explained_variance | 0.999    |
| fps                | 3397     |
| nupdates           | 270      |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0115  |
| total_timesteps    | 5868773  |
| value_loss         | 0.0145   |
---------------------------------
done
simulating  best/malli_perus_final_v119
predicting...


computing 20
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3126     |
| nupdates           | 1        |
| policy_entropy     | 1.03     |
| policy_loss        | -0.00958 |
| total_timesteps    | 0        |
| value_loss         | 0.00935  |
---------------------------------
---------------------------------
| explained_variance | 0.868    |
| fps                | 3550     |
| nupdates           | 10       |
| policy_entropy     | 1.02     |
| policy_loss        | -0.352   |
| total_timesteps    | 196353   |
| value_loss         | 1.74     |
---------------------------------
---------------------------------
| explained_variance | 0.919    |
| fps                | 3514     |
| nupdates           | 20       |
| policy_entropy     | 1.02     |
| policy_loss        | 0.67     |
| total_timesteps    | 414523   |
| value_loss         | 1.38     |
-------------------------------

...

---------------------------------
| explained_variance | 0.999    |
| fps                | 3406     |
| nupdates           | 270      |
| policy_entropy     | 1.01     |
| policy_loss        | -0.0153  |
| total_timesteps    | 5868773  |
| value_loss         | 0.0137   |
---------------------------------
done
simulating  best/malli_perus_final_v149
predicting...


computing 50
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3265     |
| nupdates           | 1        |
| policy_entropy     | 1.01     |
| policy_loss        | -0.011   |
| total_timesteps    | 0        |
| value_loss         | 0.00938  |
---------------------------------
---------------------------------
| explained_variance | 0.747    |
| fps                | 3600     |
| nupdates           | 10       |
| policy_entropy     | 0.986    |
| policy_loss        | 0.018    |
| total_timesteps    | 196353   |
| value_loss         | 3.04     |
---------------------------------
---------------------------------
| explained_variance | 0.939    |
| fps                | 3522     |
| nupdates           | 20       |
| policy_entropy     | 0.978    |
| policy_loss        | -0.1     |
| total_timesteps    | 414523   |
| value_loss         | 0.787    |
-------------------------------

---------------------------------
| explained_variance | 0.998    |
| fps                | 3467     |
| nupdates           | 270      |
| policy_entropy     | 1.02     |
| policy_loss        | -0.015   |
| total_timesteps    | 5868773  |
| value_loss         | 0.0194   |
---------------------------------
done
simulating  best/malli_perus_final_v150
predicting...


computing 51
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3183     |
| nupdates           | 1        |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0113  |
| total_timesteps    | 0        |
| value_loss         | 0.00908  |
---------------------------------
---------------------------------
| explained_variance | 0.638    |
| fps                | 3542     |
| nupdates           | 10       |
| policy_entropy     | 1.01     |
| policy_loss        | -1.67    |
| total_timesteps    | 196353   |
| value_loss         | 7.14     |
---------------------------------
---------------------------------
| explained_variance | 0.73     |
| fps                | 3520     |
| nupdates           | 20       |
| policy_entropy     | 0.982    |
| policy_loss        | 1.15     |
| total_timesteps    | 414523   |
| value_loss         | 4.78     |
-------------------------------

KeyboardInterrupt: 

In [ ]:
cc2=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc2.render_distrib(load='results/distrib_base_v',n=100,stats_results='results/distrib_stats')


# Työttömyysputken poisto

Työttömyysputkelle meneminen on usein hyvin suosittua elinkaarimalleissa. Tarkastellaan millainen työllisyysvaikutus on putken poistamisella.

In [ ]:
cc1_putki=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
poistomalli='best/putki/malli_putki'
cc1_putki.run_distrib(n=15,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=poistomalli,plot=True,cont=True,start_from=perusmalli,results='results/putki/distrib_poisto',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)


No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki False
include_pinkslip True

computing 0
train...
phase 1
batch 9 learning rate 0.125 scaled 0.375
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 2635     |
| nupdates           | 1        |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0126  |
| total_timesteps    | 0        |
| value_loss         | 0.0127   |
---------------------------------


In [ ]:
cc1_putki_b=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
cc1_putki_b.render_distrib(load='results/putki/distrib_poisto_v')
cc1_putki_b.compare_simstats('results/putki/distrib_poisto_v','results/putki/distrib_poisto_v')                    
